In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, timedelta
import utils

In [2]:
events = utils.get_events()

In [3]:
events.head(1)

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,city,region,country,device_type,screen_resolution,operating_system_version,browser_version,month,day,hora
0,2018-05-18 00:11:59,viewed product,4886f805,NaN,9288.0,Samsung Galaxy J7 Prime,Excelente,32GB,Dourado,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5,18,0


## Busco location de usuarios

In [5]:
con_city = events.loc[events.city != 'Unknown']

In [9]:
agrupadas = con_city.groupby(['person']).city.value_counts()

In [10]:
unstacked = agrupadas.unstack(fill_value=0)
unstacked.head(1)

city,Abadiania,Abaete,Abaetetuba,Abelardo Luz,Abrantes,Abreu e Lima,Acailandia,Acajutiba,Acarau,Acopiara,...,Woodbridge,Wrexham,Xambioa,Xanxere,Xavantina,Xaxim,Xexeu,Xinguara,Yellowknife,Ze Doca
person,,,,,,,,,,,,,,,,,,,,,
00091926,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
feature_location = unstacked.idxmax(axis=1)
feature_location.head()

person
00091926         Carlos Barbosa
00091a7a               Salvador
000ba417    Rio Grande da Serra
000c79fe         Belo Horizonte
000e4d9e                Igarape
dtype: object

In [12]:
feature_location.describe()

count         33156
unique         1941
top       São Paulo
freq           5419
dtype: object

In [13]:
person_con_location = feature_location.count()
person_totales = events.person.nunique()
print("Hay {} de {} personas con location asignado".format(person_con_location,person_totales))

Hay 33156 de 38829 personas con location asignado


### Agrego a events la info de city para todas estas personas que encontré

In [14]:
events.city = events.person.map(feature_location)

#### Selecciono los conversion por city

In [33]:
conversion = events.loc[events.event == 'conversion']
conversion.head(1)

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,city,region,country,device_type,screen_resolution,operating_system_version,browser_version,month,day,hora
195,2018-05-18 01:12:20,conversion,49c19e32,NaN,2683.0,iPhone 5c,Bom,16GB,Rosa,NaN,...,São Paulo,NaN,NaN,NaN,NaN,NaN,NaN,5,18,1


In [16]:
print("Conversiones por ciudad:")
conversion.city.value_counts().head()

Conversiones por ciudad:


São Paulo         1202
Salvador           313
Rio de Janeiro     305
Belo Horizonte     280
Brasília           130
Name: city, dtype: int64

In [62]:
conversions_per_city = conversion.groupby(['city']).agg({'person':'nunique'})
conversions_per_city.sort_values(by='person',ascending=False).head()

,person
city,
São Paulo,677
Rio de Janeiro,194
Belo Horizonte,178
Salvador,157
Brasília,105


In [63]:
print("Cantidad de usuarios únicos por ciudad:")
unique_person_per_city = events.groupby(['city']).agg({'person':'nunique'})
unique_person_per_city.sort_values(by='person',ascending=False).head()

Cantidad de usuarios únicos por ciudad:


,person
city,
São Paulo,5419
Rio de Janeiro,1763
Belo Horizonte,1230
Salvador,1156
Brasília,762


#### Agrego un feature relación de conversion/usuarios para cada location

In [73]:
#conversion['city'] = conversion.index
unique_person_per_city['conversion'] = unique_person_per_city.index.map(conversions_per_city.person).fillna(value=0).astype(int)
unique_person_per_city.sort_values(by='conversion',ascending=False).head()

,person,conversion,relacion
city,,,
São Paulo,5419,677,12
Rio de Janeiro,1763,194,11
Belo Horizonte,1230,178,14
Salvador,1156,157,13
Brasília,762,105,13


In [79]:
unique_person_per_city['relacion'] = unique_person_per_city.conversion / unique_person_per_city.person
unique_person_per_city.sort_values(by='relacion',ascending=False).head()

,person,conversion,relacion
city,,,
Lewes,1,1,1.0
Raposa (1),1,1,1.0
Fairfield,1,1,1.0
Fartura,1,1,1.0
Torrinha,1,1,1.0


Hasta acá tengo:
* unique_person_per_city
 * index: ciudad
 * columns:
   * person: cantidad de usuarios únicos cuya ciudad más frecuente es esa
   * conversion: cantidad de usuarios únicos de esa ciudad que hicieron una conversión
   * relacion: float [0;1] conversion/person
* feature_location (serie)
 * index: person
 
 